## Convert files from Thermo

ThermoRawFileParser https://github.com/compomics/ThermoRawFileParser

Convert profile MS1 data from Thermo raw files to centroid (MS1 and MS2) mzml

In [1]:
!mono resources/ThermoRawFileParser/ThermoRawFileParser.exe -d="data/raw/" -o="results/"

2021-10-21 16:40:05 INFO Started analyzing folder data/raw/
2021-10-21 16:40:05 INFO The folder contains 6 RAW files
2021-10-21 16:40:05 INFO Started parsing data/raw/FilipinIII.raw
2021-10-21 16:40:06 INFO Processing 3890 MS scans
10% 20% 30% 40% 50% 60% 70% 80% 90% 100% 

2021-10-21 16:40:15 INFO Finished parsing data/raw/FilipinIII.raw
2021-10-21 16:40:15 INFO Started parsing data/raw/Pentamycin.raw
2021-10-21 16:40:17 INFO Processing 3827 MS scans
10% 20% 30% 40% 50% 60% 70% 80% 90% 100% 

2021-10-21 16:40:25 INFO Finished parsing data/raw/Pentamycin.raw
2021-10-21 16:40:25 INFO Started parsing data/raw/Leupeptin.raw
2021-10-21 16:40:26 INFO Processing 3907 MS scans
10% 20% 30% 40% 50% 60% 70% 80% 90% 100% 

2021-10-21 16:40:34 INFO Finished parsing data/raw/Leupeptin.raw
2021-10-21 16:40:34 INFO Started parsing data/raw/Epemicins.raw
2021-10-21 16:40:35 INFO Processing 4611 MS scans
10% 20% 30% 40% 50% 60% 70% 80% 90% 100% 

2021-10-21 16:40:45 INFO Finished parsing data/raw/Epemi

## Copy all the files to data/raw directory and you can use the following script to generate a samples.tsv file automatically:

In [2]:
!(cd data/raw && ls *.raw > filelist.txt)
import pandas as pd
header_list = ["sample_name"]
df=pd.read_csv("data/raw/filelist.txt", names=header_list)
df["sample_name"]=df["sample_name"].replace(".raw", value="", regex=True)
df["comment"] = ""
df["MAPnumber"] = ""
df.to_csv("config/samples.tsv", sep="\t")
df

,sample_name,comment,MAPnumber
0,Epemicins,,
1,GermicidinA,,
2,GermicidinB,,


## Import libraries

In [1]:
from pyopenms import *
from pandas import DataFrame
import pandas as pd
import pyteomics
from pyteomics.openms import featurexml
import numpy as np
from pyteomics import mztab

Determination of memory status is not supported on this 
 platform, measuring for memoryleaks will never fail


# Preprocessing step

## The first preprocessing function consists of six steps:

### 1) PrecursorCorrection (To the "highest intensity MS1 peak")

This algorithm is used directly after the file introduction, in order to correct any wrong MS1 precursor annotation - this means it will correct the precursos m/z in a given mass range (e.g. 0.1 Da) and then use the highest intensity peak as the precursor. We assume that, in the given mass window, the precursor with the hightest intensity was actually fragmented (top-n method), which is a method used by default in the Thermo Orbitrap instrument (Center for Biosustainability).

### OpenMS definition:

"Selection of the peak with the highest intensity as corrected precursor mass in a given mass range (e.g. precursor mass +/- 0.2 Da)

For each MS2 spectrum the corresponding MS1 spectrum is determined by using the rt information of the precursor. In the MS1, the peak with the highest intensity in a given mass range to the uncorrected precursor m/z is selected and used as corrected precursor m/z."

#### Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/release/latest/html//classOpenMS_1_1PrecursorCorrection.html#a8acf85ba8b9f249de0369bb083355982 

###### Citation: Röst, H.L., Sachsenberg, T., Aiche, S., Bielow, C., Weisser, H., Aicheler, F., Andreotti, S., Ehrlich, H.-C., Gutenbrunner, P., Kenar, E., Liang, X., Nahnsen, S., Nilse, L., Pfeuffer, J., Rosenberger, G., Rurik, M., Schmitt, U., Veit, J., Walzer, M., Wojnar, D., Wolski, W.E.,Schilling, O., Choudhary, J.S., Malmström, L., Aebersold, R., Reinert, K., Kohlbacher, O. OpenMS: A flexible open-source software platform for mass spectrometry data analysis. Nature Methods, vol. 13, 2016. doi:10.1038/nmeth.3959

In [ ]:
example_filename= "results/Epemicins/interim/Epemicins.mzML" #introduce the mzML file that you want to preprocess here. If it is more than one, refer to the other jupyter notebook
exp = MSExperiment()
MzMLFile().load(example_filename, exp)
exp.sortSpectra(True)
    
delta_mzs= []
mzs = []
rts= []
PrecursorCorrection.correctToHighestIntensityMS1Peak(exp, 100.0, True, delta_mzs, mzs, rts)
MzMLFile().store("results/Epemicins/interim/precursorcorrected_Epemicins.mzML", exp)

### 2) Mass trace detection

A mass trace extraction method that gathers peaks similar in m/z and moving along retention time.

Peaks of a MSExperiment are sorted by their intensity and stored in a list of potential chromatographic apex positions. Only peaks that are **above the noise threshold** (user-defined) are analyzed and only peaks that are n times above this minimal threshold are considered as apices. This saves computational resources and decreases the noise in the resulting output.

Starting with these, mass traces are extended in retention time. During this extension phase, the centroid m/z is computed on-line as an intensity-weighted mean of peaks.

The extension phase ends when either the frequency of gathered peaks drops below a threshold (min_sample_rate, see MassTraceDetection parameters) or when the number of missed scans exceeds a threshold (trace_termination_outliers, see MassTraceDetection parameters).

Finally, only mass traces that pass a filter (a certain minimal and maximal length as well as having the minimal sample rate criterion fulfilled) get added to the result."

#### Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/nightly/html/classOpenMS_1_1MassTraceDetection.html#abff6e392ce6da7af8f083397494a7971

###### Citation: Röst, H.L., Sachsenberg, T., Aiche, S., Bielow, C., Weisser, H., Aicheler, F., Andreotti, S., Ehrlich, H.-C., Gutenbrunner, P., Kenar, E., Liang, X., Nahnsen, S., Nilse, L., Pfeuffer, J., Rosenberger, G., Rurik, M., Schmitt, U., Veit, J., Walzer, M., Wojnar, D., Wolski, W.E.,Schilling, O., Choudhary, J.S., Malmström, L., Aebersold, R., Reinert, K., Kohlbacher, O. OpenMS: A flexible open-source software platform for mass spectrometry data analysis. Nature Methods, vol. 13, 2016. doi:10.1038/nmeth.3959

In [5]:
mass_traces = []
mtd = MassTraceDetection()
mtd_par = mtd.getDefaults()
mtd_par.setValue("mass_error_ppm", 10.0) 
mtd_par.setValue("noise_threshold_int", 1.0e04)
mtd.setParameters(mtd_par)
mtd.run(exp, mass_traces, 0)

Progress of 'mass trace detection':
-- done [took 0.44 s (CPU), 0.43 s (Wall)] -- 


### 3) Elution peak detection

"Extracts chromatographic peaks from a mass trace.

Mass traces may consist of several consecutively (partly overlapping) eluting peaks, e.g., stemming from (almost) isobaric compounds that are separated by retention time. Especially in metabolomics, isomeric compounds with exactly the same mass but different retentional behaviour may still be contained in the same mass trace.

This method first applies smoothing on the mass trace's intensities, then detects local minima/maxima in order to separate the chromatographic peaks from each other. Detection of maxima is performed on the smoothed intensities and uses a fixed peak width (given as parameter chrom_fwhm) within which only a single maximum is expected. Currently smoothing is done using SavitzkyGolay smoothing with a second order polynomial and a frame length of the fixed peak width.

Depending on the "width_filtering" parameters, mass traces are filtered by length in seconds ("fixed" filter) or by quantile.

The output of the algorithm is a set of chromatographic peaks for each mass trace, i.e. a vector of split mass traces (see ElutionPeakDetection parameters).

In general, a user would want to call the "detectPeaks" functions, potentially followed by the "filterByPeakWidth" function."

This method in other words is "deconvolution".

#### Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/nightly/html/classOpenMS_1_1ElutionPeakDetection.html#details

###### Citation: Röst, H.L., Sachsenberg, T., Aiche, S., Bielow, C., Weisser, H., Aicheler, F., Andreotti, S., Ehrlich, H.-C., Gutenbrunner, P., Kenar, E., Liang, X., Nahnsen, S., Nilse, L., Pfeuffer, J., Rosenberger, G., Rurik, M., Schmitt, U., Veit, J., Walzer, M., Wojnar, D., Wolski, W.E.,Schilling, O., Choudhary, J.S., Malmström, L., Aebersold, R., Reinert, K., Kohlbacher, O. OpenMS: A flexible open-source software platform for mass spectrometry data analysis. Nature Methods, vol. 13, 2016. doi:10.1038/nmeth.3959

In [6]:
mass_traces_split = []
mass_traces_final = []
epd = ElutionPeakDetection()
epd_par = epd.getDefaults()
epd_par.setValue("width_filtering", "fixed")
epd.setParameters(epd_par)
epd.detectPeaks(mass_traces, mass_traces_split)
    
if (epd.getParameters().getValue("width_filtering") == "auto"):
     epd.filterByPeakWidth(mass_traces_split, mass_traces_final)
else:
    mass_traces_final = mass_traces_split

Progress of 'elution peak detection':
-- done [took 2.18 s (CPU), 0.31 s (Wall)] -- 


### 4) Feature detection

FeatureFinderMetabo assembles metabolite features from singleton mass traces.

Mass traces alone would allow for further analysis such as metabolite ID or statistical evaluation. However, in general, monoisotopic mass traces are accompanied by satellite C13 peaks and thus may render the analysis more difficult. **FeatureFinderMetabo fulfills a further data reduction step by assembling compatible mass traces to metabolite features** (that is, all mass traces originating from one metabolite). To this end, multiple metabolite hypotheses are formulated and scored according to how well differences in RT (optional), m/z or intensity ratios match to those of theoretical isotope patterns.

Detailed information is stored in the featureXML output: it contains meta-values for each feature about the mass trace differences (inspectable via TOPPView). 

By default, the linear model is used (better for small molecules - use the 13C for proteomics).

#### Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/nightly/html/TOPP_FeatureFinderMetabo.html

###### Citation: Röst, H.L., Sachsenberg, T., Aiche, S., Bielow, C., Weisser, H., Aicheler, F., Andreotti, S., Ehrlich, H.-C., Gutenbrunner, P., Kenar, E., Liang, X., Nahnsen, S., Nilse, L., Pfeuffer, J., Rosenberger, G., Rurik, M., Schmitt, U., Veit, J., Walzer, M., Wojnar, D., Wolski, W.E.,Schilling, O., Choudhary, J.S., Malmström, L., Aebersold, R., Reinert, K., Kohlbacher, O. OpenMS: A flexible open-source software platform for mass spectrometry data analysis. Nature Methods, vol. 13, 2016. doi:10.1038/nmeth.3959

In [7]:
feature_map_FFM = FeatureMap()
feat_chrom = []
ffm = FeatureFindingMetabo()
ffm_par = ffm.getDefaults() 
ffm_par.setValue("isotope_filtering_model", "none")
ffm_par.setValue("remove_single_traces", "false")
ffm_par.setValue("mz_scoring_by_elements", "false")
ffm_par.setValue("report_convex_hulls", "true")
ffm.setParameters(ffm_par)
ffm.run(mass_traces_final, feature_map_FFM, feat_chrom)
feature_map_FFM.setUniqueIds()
fh = FeatureXMLFile()
filename= "results/Epemicins/interim/precursorcorrected_Epemicins.mzML"
feature_map_FFM.setPrimaryMSRunPath([filename.encode()])
fh.store('./results/Epemicins/interim/FFM_Epemicins.featureXML', feature_map_FFM)

Progress of 'assembling mass traces to features':
-- done [took 4.40 s (CPU), 0.71 s (Wall)] -- 


### 5) Metabolite adduct decharger (MetaboliteFeatureDeconvolution)

For each peak, this algorithm reconstructs neutral masses by enumerating all possible adducts with matching charge. Provide the probability of adduct formation according to your instrument using the parameter "potential adducts".

With SIRIUS, you are only able to use singly charged adducts so charges higher than 1 are filtered out.

#### Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/nightly/html/UTILS_MetaboliteAdductDecharger.html

###### Citation: Röst, H.L., Sachsenberg, T., Aiche, S., Bielow, C., Weisser, H., Aicheler, F., Andreotti, S., Ehrlich, H.-C., Gutenbrunner, P., Kenar, E., Liang, X., Nahnsen, S., Nilse, L., Pfeuffer, J., Rosenberger, G., Rurik, M., Schmitt, U., Veit, J., Walzer, M., Wojnar, D., Wolski, W.E.,Schilling, O., Choudhary, J.S., Malmström, L., Aebersold, R., Reinert, K., Kohlbacher, O. OpenMS: A flexible open-source software platform for mass spectrometry data analysis. Nature Methods, vol. 13, 2016. doi:10.1038/nmeth.3959

In [8]:
mfd = MetaboliteFeatureDeconvolution()
mdf_par = mfd.getDefaults()
mdf_par.setValue("potential_adducts",  [b"H:+:0.6",b"Na:+:0.2",b"NH4:+:0.1", b"H2O:-:0.1"])
mdf_par.setValue("charge_min", 1, "Minimal possible charge")
mdf_par.setValue("charge_max", 1, "Maximal possible charge")
mdf_par.setValue("charge_span_max", 1)
mdf_par.setValue("max_neutrals", 1)
mfd.setParameters(mdf_par)
    
feature_map_DEC = FeatureMap()
cons_map0 = ConsensusMap()
cons_map1 = ConsensusMap()
mfd.compute(feature_map_FFM, feature_map_DEC, cons_map0, cons_map1)
fxml = FeatureXMLFile()
fh.store('./results/Epemicins/interim/MFD_Epemicins.featureXML', feature_map_DEC)

MassExplainer table size: 12
4611 spectra and 1 chromatograms stored.
Generating Masses with threshold: -6.90776 ...
done
0 of 2852 valid net charge compomer results did not pass the feature charge constraints
Inferring edges raised edge count from 2852 to 2852
Found 2852 putative edges (of 131864) and avg hit-size of 1
Using solver 'coinor' ...
Optimal solution found!
<Using solver 'coinor' ...> occurred 7 times
 Branch and cut took 0.919725 seconds,  with objective value: 6.88.
<Optimal solution found!> occurred 7 times
ILP score is: 6.88
Agreeing charges: 541/3526
ConsensusFeature::computeDechargeConsensus() WARNING: Feature's charge is 0! This will lead to M=0!


Compomer: Da -41.0003; q_net 0; logP -3.91202[[ (H2O1Na1) --> () ]]
 Compomer: Da -36.0449; q_net 0; logP -4.60517[[ (H2O1H4N1) --> () ]]
 Compomer: Da -21.9819; q_net 0; logP -2.12026[[ (Na1) --> (H1) ]]
 Compomer: Da -19.0184; q_net 0; logP -2.81341[[ (H1H2O1) --> () ]]
 Compomer: Da -17.0265; q_net 0; logP -2.81341[[ (H4N1) --> (H1) ]]
 Compomer: Da -4.9554; q_net 0; logP -3.91202[[ (Na1) --> (H4N1) ]]
 Compomer: Da 4.9554; q_net 0; logP -3.91202[[ (H4N1) --> (Na1) ]]
 Compomer: Da 17.0265; q_net 0; logP -2.81341[[ (H1) --> (H4N1) ]]
 Compomer: Da 19.0184; q_net 0; logP -2.81341[[ () --> (H1H2O1) ]]
 Compomer: Da 21.9819; q_net 0; logP -2.12026[[ (H1) --> (Na1) ]]
 Compomer: Da 36.0449; q_net 0; logP -4.60517[[ () --> (H2O1H4N1) ]]
 Compomer: Da 41.0003; q_net 0; logP -3.91202[[ () --> (H2O1Na1) ]]
 

In [3]:
from collections import defaultdict
from functools import reduce
from pathlib import Path
from time import perf_counter
import sys

from IPython.core.display import display
from pandas import CategoricalDtype
import numpy as np
from pyopenms import *
import pandas as pd
import os

class FeatureMapDF(FeatureMap):      
    def __init__(self):
        super().__init__()

    def get_df(self):
        def gen(fmap: FeatureMap, fun):
            for f in fmap:
                yield from fun(f)

        def extractMetaData(f: Feature):
            metavals = []
            types = []
            for k in metavals:
                if k == b"num_of_masstraces":
                    types.append('f')
                elif k== b"FWHM":
                    types.append('f')
                elif k== b"max_height":
                    types.append('i4')
                elif k== b"masstrace_intensity":
                    types.append("f")
            # subfeatures = f.getFeatureList()  # type: list[FeatureHandle]
            pep = f.getPeptideIdentifications()  # type: list[PeptideIdentification]
            bb = f.getConvexHull().getBoundingBox2D()
            for u in metavals:
                f.getMetaValue(u)
            if len(pep) != 0:
                hits = pep[0].getHits()
                if len(hits) != 0:
                    besthit = hits[0]  # type: PeptideHit
                    yield f.getUniqueId(), besthit.getSequence().toString(), f.getCharge(), f.getRT(), f.getMZ(), bb[0][0], bb[1][0], f.getMetaValue("num_of_masstraces"), f.getOverallQuality(), f.getIntensity()
                else:
                    yield f.getUniqueId(), None, f.getCharge(), f.getRT(), f.getMZ(), bb[0][0], bb[1][0], f.getMetaValue("num_of_masstraces"), f.getOverallQuality(), f.getIntensity()
            else:
                yield f.getUniqueId(), None, f.getCharge(), f.getRT(), f.getMZ(), bb[0][0], bb[1][0], f.getMetaValue("num_of_masstraces"), f.getOverallQuality(), f.getIntensity()

        cnt = self.size()

        mddtypes = [('id', np.dtype('uint64')), ('sequence', 'U200'), ('charge', 'i4'), ('RT', 'f'), ('mz', 'f'),
                    ('RTstart', 'f'), ('RTend', 'f'), ("num_of_masstraces", 'f'),
                    ('quality', 'f'), ('intensity', 'f')]
        mdarr = np.fromiter(iter=gen(self, extractMetaData), dtype=mddtypes, count=cnt)
        df= pd.DataFrame(mdarr).set_index('id').sort_values("mz").drop(columns= "sequence")
        df= df[df["num_of_masstraces"]>=2]
        return df

In [8]:
filename= './results/Epemicins/interim/MFD_Epemicins.featureXML'
fmap = FeatureMapDF()
FeatureXMLFile().load(filename, fmap)
DF= fmap.get_df()
feature_csv= './results/Epemicins/features_Epemicins.csv'
DF.to_csv(feature_csv)
display(DF)

,charge,RT,mz,RTstart,RTend,num_of_masstraces,quality,intensity
id,,,,,,,,
15695211054020622558,1,329.607452,208.133240,326.792847,335.102264,2.0,0.000180,3.212667e+07
11553075680875761771,1,310.614807,208.133255,303.933105,315.405701,2.0,0.000011,1.985221e+06
16931706070924112617,1,379.585754,260.185669,362.698792,455.897400,2.0,0.000258,4.589339e+07
2464053971863077264,1,228.511551,261.123413,222.785919,235.216217,2.0,0.000025,4.301382e+06
14207239120303140199,1,316.367584,271.060028,302.927643,321.092896,2.0,0.000009,1.677704e+06
...,...,...,...,...,...,...,...,...
13812678751659148901,1,350.433289,1603.895020,348.765778,355.269623,3.0,0.001192,1.190649e+08
4289608483695663489,1,342.791321,1604.396851,337.522614,352.043243,2.0,0.000371,4.679359e+07
7662137660621363607,2,47.941818,1639.049805,43.264481,61.428474,2.0,0.000003,2.294459e+05


### 6) PrecursorCorrection (To the "nearest feature”)

This algorithm is used after feature detection, adduct grouping and even identification via accurate mass search. It basically allows the precursor correction on MS2 level. 

Which means that if there are MS2 spectra in my feature space which have been measured in isotope traces, it “corrects” the MS2 spectrum annotation to the monoisotopic trace. That is why you have a high mass deviation 100 pm, but 0.0 rt tolerance. So it basically corrects the MS2 to the feature centroid that can be found/mapped by SIRIUS::preprocessing later on.

Wrong assignment of the mono-isotopic mass for precursors are assumed:

1. if precursor_mz matches the mz of a non-monoisotopic feature mass trace and 
2. in the case that believe_charge is true: if feature_charge matches the precursor_charge In the case of wrong mono-isotopic assignment several options for correction are available: keep_original will create a copy of the precursor and tandem spectrum for the new mono-isotopic mass trace and retain the original one. all_matching_features does this not for only the closest feature but all features in a question.

#### Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/nightly/html/classOpenMS_1_1PrecursorCorrection.html

###### Citation: Röst, H.L., Sachsenberg, T., Aiche, S., Bielow, C., Weisser, H., Aicheler, F., Andreotti, S., Ehrlich, H.-C., Gutenbrunner, P., Kenar, E., Liang, X., Nahnsen, S., Nilse, L., Pfeuffer, J., Rosenberger, G., Rurik, M., Schmitt, U., Veit, J., Walzer, M., Wojnar, D., Wolski, W.E.,Schilling, O., Choudhary, J.S., Malmström, L., Aebersold, R., Reinert, K., Kohlbacher, O. OpenMS: A flexible open-source software platform for mass spectrometry data analysis. Nature Methods, vol. 13, 2016. doi:10.1038/nmeth.3959

In [ ]:
PrecursorCorrection.correctToNearestFeature(feature_map_DEC, exp, 0.0, 100.0, True, False, False, False, 3, 0)

set()

HighResPrecursorMassCorrector warning: at least one feature has no convex hull - omitting feature for matching


### SIRIUS Adapter
The SIRIUS function is optional and includes the SIRIUS Adapter Algorithm from the Boecher lab. 


The algorithm generates formula prediction from scores calculated from 1) MS2 fragmentation scores (ppm error + intensity) and 2) MS1 isotopic pattern scores.


It can only compute feautures that are singly charged. There is also a timeout for compounds (compound timeout so that it doesn't compute for longer than 100 seconds per feature, which normally happens with larger molecules).


-sirius:compound_timeout <number>                    
Maximal computation time in seconds for a single compound. 0 for an infinite amount of time. (default: '100' min: '0')

    
This algorith can help in data dereplication and analysis for direct library search. 

#### Documentation: https://boecker-lab.github.io/docs.sirius.github.io/

###### Citation: Kai Dührkop, Markus Fleischauer, Marcus Ludwig, Alexander A. Aksenov, Alexey V. Melnik, Marvin Meusel, Pieter C. Dorrestein, Juho Rousu, and Sebastian Böcker, SIRIUS 4: Turning tandem mass spectra into metabolite structure information. Nature Methods 16, 299–302, 2019.

We need to preprocess (FeatureFinderMetabo) again without convex hulls this time, otherwise the results from SIRIUS are incorrect

In [ ]:
feature_map_FFM = FeatureMap()
feat_chrom = []
ffm = FeatureFindingMetabo()
ffm_par = ffm.getDefaults() 
ffm_par.setValue("isotope_filtering_model", "none")
ffm_par.setValue("remove_single_traces", "false")
ffm_par.setValue("mz_scoring_by_elements", "false")
ffm.setParameters(ffm_par)
ffm.run(mass_traces_final, feature_map_FFM, feat_chrom)
feature_map_FFM.setUniqueIds()
fh = FeatureXMLFile()
filename= "results/Epemicins/interim/precursorcorrected_Epemicins.mzML"
feature_map_FFM.setPrimaryMSRunPath([filename.encode()])
fh.store('./results/Epemicins/interim/FFM_noconvexhulls_Epemicins.featureXML', feature_map_FFM)
mfd = MetaboliteFeatureDeconvolution()
mdf_par = mfd.getDefaults()
mdf_par.setValue("potential_adducts",  [b"H:+:0.6",b"Na:+:0.2",b"NH4:+:0.1", b"H2O:-:0.1"])
mdf_par.setValue("charge_min", 1, "Minimal possible charge")
mdf_par.setValue("charge_max", 1, "Maximal possible charge")
mdf_par.setValue("charge_span_max", 1)
mdf_par.setValue("max_neutrals", 1)
mfd.setParameters(mdf_par)
feature_map_DEC = FeatureMap()
cons_map0 = ConsensusMap()
cons_map1 = ConsensusMap()
mfd.compute(feature_map_FFM, feature_map_DEC, cons_map0, cons_map1)
fxml = FeatureXMLFile()
fh.store('./results/Epemicins/interim/MFD_noconvexhulls_Epemicins.featureXML', feature_map_DEC)

In [ ]:
sirius_algo = SiriusAdapterAlgorithm()
sirius_algo_par = sirius_algo.getDefaults()
sirius_algo_par.setValue("preprocessing:filter_by_num_masstraces", 2) #Number of mass traces each feature has to have to be included
sirius_algo_par.setValue("preprocessing:feature_only", "true") #Uses the feature information from in_featureinfo to reduce the search space to MS2
sirius_algo_par.setValue("sirius:profile", "orbitrap")
sirius_algo_par.setValue("sirius:db", "none")
sirius_algo_par.setValue("sirius:ions_considered", "[M+H]+, [M-H2O+H]+, [M+Na]+, [M+NH4]+")
sirius_algo_par.setValue("sirius:elements_enforced", "CHN[15]OS[4]") 
sirius_algo_par.setValue("project:processors", 2)
sirius_algo_par.setValue("fingerid:db", "BIO")
sirius_algo.setParameters(sirius_algo_par)
    
featureinfo = "./results/Epemicins/interim/MFD_noconvexhulls_Epemicins.featureXML"
fm_info = FeatureMapping_FeatureMappingInfo()
feature_mapping = FeatureMapping_FeatureToMs2Indices() 
sirius_algo.preprocessingSirius(featureinfo,
                                exp,
                                fm_info,
                                feature_mapping)
sirius_algo.logFeatureSpectraNumber(featureinfo, 
                                    feature_mapping,
                                    exp)
msfile = SiriusMSFile()
debug_level = 3
sirius_tmp = SiriusTemporaryFileSystemObjects(debug_level)
siriusstring= String(sirius_tmp.getTmpMsFile())
feature_only = sirius_algo.isFeatureOnly()
isotope_pattern_iterations = sirius_algo.getIsotopePatternIterations()
no_mt_info = sirius_algo.isNoMasstraceInfoIsotopePattern()
compound_info = []
msfile.store(exp,
             String(sirius_tmp.getTmpMsFile()),
             feature_mapping, 
             feature_only,
             isotope_pattern_iterations, 
             no_mt_info, 
             compound_info)
out_csifingerid = "./results/Epemicins/interim/structures_Epemicins.mzTab" 
executable= "/path/to/sirius_executable"
subdirs = sirius_algo.callSiriusQProcess(String(sirius_tmp.getTmpMsFile()),
                                         String(sirius_tmp.getTmpOutDir()),
                                         String(executable),
                                         String(out_csifingerid),
                                         False)
candidates = sirius_algo.getNumberOfSiriusCandidates()
sirius_result = MzTab()
siriusfile = MzTabFile()
SiriusMzTabWriter.read(subdirs,
                       filename,
                       candidates,
                       sirius_result)
siriusfile.store("./results/Epemicins/interim/formulas_Epemicins.mzTab", sirius_result) 

In [7]:
siriusfile= "./results/Epemicins/interim/formulas_Epemicins.mzTab"
sirius=  pyteomics.mztab.MzTab(siriusfile, encoding='UTF8', table_format='df')
sirius.metadata
df= sirius.small_molecule_table
SIRIUS_DF= df.drop(columns= ["identifier", "smiles", "inchi_key", "description", "calc_mass_to_charge", "charge", "taxid", "species","database", "database_version", "spectra_ref", "search_engine", "modifications"])
SIRIUS_DF.to_csv("./results/Epemicins/formulas_Epemicins.csv")
SIRIUS_DF= pd.read_csv("./results/Epemicins/formulas_Epemicins.csv", index_col= "Unnamed: 0")
SIRIUS_DF=SIRIUS_DF[SIRIUS_DF["opt_global_explainedIntensity"] >= 0.6] #opt_global_explainedIntensity should be higher than 0.8 or 0.9 even for reliable results
SIRIUS_DF= SIRIUS_DF.sort_values(by= "exp_mass_to_charge")
SIRIUS_DF

,chemical_formula,exp_mass_to_charge,retention_time,best_search_engine_score[1],best_search_engine_score[2],best_search_engine_score[3],opt_global_adduct,opt_gobal_precursorFormula,opt_global_rank,opt_global_explainedPeaks,opt_global_explainedIntensity,opt_global_median_mass_error_fragment_peaks_ppm,opt_global_median_absolute_mass_error_fragment_peaks_ppm,opt_global_mass_error_precursor_ppm,opt_global_compoundId,opt_global_compoundScanNumber,opt_global_featureId,opt_global_native_id
2782,C7H16N3OS,208.133234,329.607460,13.604441,13.604441,0.000000,[M + H4 + N]+,C7H20N3OS,3,4,0.935599,6.190684,8.389324,-9.606428,1576,1577,id_11804993995459637598,controllerType=0 controllerNumber=1 scan=1577|...
2781,C10H12N3O,208.133234,329.607460,15.230887,14.193324,1.037563,[M + H4 + N]+,C10H16N3O,2,4,0.935599,6.881382,6.881382,6.590543,1576,1577,id_11804993995459637598,controllerType=0 controllerNumber=1 scan=1577|...
3687,C14H13NO3,261.123425,228.511545,46.917919,46.549750,0.368169,[M + H4 + N]+,C14H17NO3,1,11,0.833473,-1.250390,1.250390,0.215089,1057,1058,id_14669583106630530835,controllerType=0 controllerNumber=1 scan=1058
3688,C12H11N4O2,261.123425,228.511545,39.257591,39.257591,0.000000,[M + H4 + N]+,C12H15N4O2,2,14,0.996021,5.349610,5.349610,5.357041,1057,1058,id_14669583106630530835,controllerType=0 controllerNumber=1 scan=1058
3689,C7H13N7OS,261.123425,228.511545,34.713566,34.713566,0.000000,[M + H4 + N]+,C7H17N7OS,3,13,0.982498,7.144945,7.144945,-2.411102,1057,1058,id_14669583106630530835,controllerType=0 controllerNumber=1 scan=1058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
551,C45H22N12O24S10,1451.827148,415.963700,29.538484,29.538484,0.000000,[M + H4 + N]+,C45H26N12O24S10,1,12,0.756276,0.469991,2.411584,-9.902878,2076,2077,id_16989201925219475051,controllerType=0 controllerNumber=1 scan=2077
555,C31H26N10O40S8,1451.827148,415.963700,29.538484,29.538484,0.000000,[M + H4 + N]+,C31H30N10O40S8,5,12,0.756276,0.469991,2.411584,-9.657702,2076,2077,id_16989201925219475051,controllerType=0 controllerNumber=1 scan=2077
554,C38H38N10O20S15,1451.827148,415.963700,29.538484,29.538484,0.000000,[M + H4 + N]+,C38H42N10O20S15,4,12,0.756276,0.469991,2.411584,-9.730014,2076,2077,id_16989201925219475051,controllerType=0 controllerNumber=1 scan=2077
552,C30H42N12O22S16,1451.827148,415.963700,29.538484,29.538484,0.000000,[M + H4 + N]+,C30H46N12O22S16,2,12,0.756276,0.469991,2.411584,-9.281178,2076,2077,id_16989201925219475051,controllerType=0 controllerNumber=1 scan=2077


### CSIFinger:ID

The CSI_fingerID function is another algorithm from the Boecher lab, just like SIRIUS adapter and is using the formula predictions from SIRIUS, to search in structural libraries and predict the structure of each formula

#### Documentation: https://boecker-lab.github.io/docs.sirius.github.io/

##### Citation: Kai Dührkop, Huibin Shen, Marvin Meusel, Juho Rousu, and Sebastian Böcker, Searching molecular structure databases with tandem mass spectra using CSI:FingerID, PNAS October 13, 2015 112 (41) 12580-12585, https://doi.org/10.1073/pnas.1509788112

In [ ]:
csi_file.store("./results/Epemicins/interim/structures_Epemicins.mzTab", csi_result)
csi_file= "./results/Epemicins/interim/structures_Epemicins.mzTab"
CSI=  pyteomics.mztab.MzTab(csi_file, encoding='UTF8', table_format='df')
CSI.metadata
df= CSI.small_molecule_table
csifingerID= df.drop(columns= ["calc_mass_to_charge", "charge", "taxid", "species","database", "database_version", "spectra_ref", "search_engine", "modifications"])
csifingerID.to_csv("./results/Epemicins/structures_Epemicins.csv)

### Explanation of columns
##### mz= mass-to-charge ratio (m/z)
##### RT= retention time (sec)
##### intensity = intensity of the feature (AU-arbitrary units)
##### FWHM= Full Width of the peak at Half its Maximum height
##### num_of_masstraces	= number of mass traces detected (single mass traces are excluded). This is relevant to the isotopic pattern
##### isotope_distances = distance in mz between the isotopes (jumps of app. 1 is important to confirm that this is a real feature) 

In [8]:
#example
csifingerID= pd.read_csv("./results/Epemicins/structures_Epemicins.csv",index_col= "Unnamed: 0")
csifingerID

,identifier,chemical_formula,smiles,inchi_key,description,exp_mass_to_charge,retention_time,best_search_engine_score[1],opt_global_rank,opt_global_compoundId,opt_global_compoundScanNumber,opt_global_featureId,opt_global_native_id,opt_global_adduct,opt_global_dblinks,opt_global_dbflags
0,14852549|14852550|25053655|24971283|53652498|5...,C25H45NO5,CCCCCCCCCCCC(CC1C(C(=O)O1)CCCCCC)OC(=O)CNC=O,ANPULBVRORHAGO,"Orlistat derivative, 15g",457.363953,327.701974,-167.839042,1,1571,1572,id_6254478322423518528,controllerType=0 controllerNumber=1 scan=1572,[M + H4 + N]+,MeSH:(14852550)|PubChem:(14852549 14852550 250...,70
1,9889418|11059239,C25H45NO5,CCCCCCCCCCC1C(OC1=O)CC(CCCCCCC)OC(=O)CNC=O,RQFZMIXBLJEUGS,NaN,457.363953,327.701974,-168.290573,2,1571,1572,id_6254478322423518528,controllerType=0 controllerNumber=1 scan=1572,[M + H4 + N]+,COCONUT:(CNP0260950)|KNApSAcK:(16423)|Natural ...,3178514
2,71464556|71464557,C25H45NO5,CCCCCC=CCC=CCCCCCC(CC(=O)OC(CC(=O)O)C[N+](C)(C...,WQYXCASYXUFNSI,3-hydroxylinoleylcarnitine,457.363953,327.701974,-170.979093,3,1571,1572,id_6254478322423518528,controllerType=0 controllerNumber=1 scan=1572,[M + H4 + N]+,PubChem:(71464556 71464557)|CHEBI:(73075),34
3,9824494,C25H45NO5,CCCCCCCC(CC1C(C(=O)O1)CCCCCCCC(C)C)OC(=O)CNC=O,SMEWLQMERHEMJK,NaN,457.363953,327.701974,-171.211550,4,1571,1572,id_6254478322423518528,controllerType=0 controllerNumber=1 scan=1572,[M + H4 + N]+,COCONUT:(CNP0157254)|Natural Products:(UNPD174...,3178498
4,3049003,C25H45NO5,CCCCCCC(C)(CCN1C(C(=O)C(C1=O)(C)C)CCCCCCC(=O)O...,OHQKFFPFFFUNJW,"Ethyl 4,4-dimethyl-3,5-dioxo-1-(3-hydroxy-3-me...",457.363953,327.701974,-250.749662,5,1571,1572,id_6254478322423518528,controllerType=0 controllerNumber=1 scan=1572,[M + H4 + N]+,PubChem:(3049003)|PubChem class - safety and t...,67108866
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2572,288197|2754113|7056395|7056399|11868690|118686...,C19H28O5,CC12CCC3C(C1CCC2=O)CCC(C3(C)CC(=O)O)CC(=O)O,CEUQHWSIXGFNGI,NaN,337.199054,465.541832,-155.792453,1,2336,2337,id_361053789030700068,controllerType=0 controllerNumber=1 scan=2337,[M + H]+,COCONUT:(CNP0068730)|PubChem:(288197 2754113 7...,3153922
2573,"""""",C19H28O5,CC(=O)C(=O)C1(CCC2C(C(=O)CCC2(C1CC(=O)O)C)(C)C)C,QWLGLRAGSFVBDI,NaN,337.199054,465.541832,-168.039526,2,2336,2337,id_361053789030700068,controllerType=0 controllerNumber=1 scan=2337,[M + H]+,COCONUT:(CNP0136763)|Natural Products:(UNPD195...,3178496
2574,354385|494326|23425264,C19H28O5,CC(=O)OC1CC2CC3(C(CC(C3C2(C1=C)O)(C)C)OC(=O)C)C,CLIPYJQRTQNGAM,".delta.-(sup 9,8.beta., 10.alpha.-triol, 3,8-d...",337.199054,465.541832,-203.768295,3,2336,2337,id_361053789030700068,controllerType=0 controllerNumber=1 scan=2337,[M + H]+,COCONUT:(CNP0216092)|Natural Products:(UNPD156...,3178498
2575,"""""",C19H28O5,CC(=O)OC1CC2CC3(CC(C(C3C2(C1=C)O)(C)C)OC(=O)C)C,KEVCQSXXJWIVSR,NaN,337.199054,465.541832,-205.873518,4,2336,2337,id_361053789030700068,controllerType=0 controllerNumber=1 scan=2337,[M + H]+,COCONUT:(CNP0261220)|Natural Products:(UNPD183...,2129920


### Create a metadata csv file for GNPS from the samples.tsv file 

In [10]:
path= "results/GNPSexport/"
isExist= os.path.exists(path)
if not isExist:
    os.mkdir("results/GNPSexport/")
df= pd.read_csv("config/samples.tsv", sep= "\t", index_col= "Unnamed: 0")
metadata= df.rename(columns= {"sample_name": "filename", "comment": "ATTRIBUTE_comment", "MAPnumber": "ATTRIBUTE_MAPnumber"})
metadata["filename"]= metadata["filename"].astype(str) +".mzml"
metadata['ATTRIBUTE_MAPnumber'] = np.arange(len(metadata))
metadata["ATTRIBUTE_comment"]= "MAP" + metadata["ATTRIBUTE_MAPnumber"].astype(str)
metadata= metadata.drop(columns= "ATTRIBUTE_MAPnumber")
metadata['ATTRIBUTE_genomeID']=metadata['filename'].str.extract(r'(NBC_?\d*)')
metadata['ATTRIBUTE_genomeIDMDNA']=metadata['filename'].str.extract(r'(MDNA_WGS_?\d*)')
metadata['ATTRIBUTE_genomeID']=metadata['ATTRIBUTE_genomeID'].fillna(metadata['ATTRIBUTE_genomeIDMDNA'])
metadata=metadata.drop(columns="ATTRIBUTE_genomeIDMDNA")
#metadata['ATTRIBUTE_genomeID']= metadata['ATTRIBUTE_genomeID'].replace(to_replace= r'NBC', value= 'NBC_', regex= True)
#metadata['ATTRIBUTE_genomeID']= metadata['ATTRIBUTE_genomeID'].replace(to_replace= r'MDNAWGS', value= 'MDNA_WGS_', regex= True)
metadata.to_csv("results/GNPSexport/metadata.tsv", sep='\t')
metadata

,filename,ATTRIBUTE_comment,ATTRIBUTE_genomeID
0,Epemicins.mzml,MAP0,NaN
1,FilipinIII.mzml,MAP1,NaN
2,GermicidinA.mzml,MAP2,NaN
3,GermicidinB.mzml,MAP3,NaN
4,Leupeptin.mzml,MAP4,NaN
5,Pentamycin.mzml,MAP5,NaN


From the CLI, I am using the following algorithms to generate files compatible to GNPS for FBMN: 

#### MapAlignerPoseClustering is used to perform a linear retention time alignment, basically correct for linear shifts in retention time. 
Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/release/latest/html/TOPP_MapAlignerPoseClustering.html

    rule MapAlignerPoseClustering:
        input:
            expand("results/{samples}/interim/MFD_{samples}.featureXML", samples=SAMPLES)
        output:
            var1= expand("results/Consensus/interim/MapAlignerPoseClustering_{samples}.featureXML", samples=SAMPLES),
            var2= expand("results/Consensus/interim/MapAlignerPoseClustering_{samples}.trafoXML", samples=SAMPLES)
        shell:
            """
            resources/OpenMS-2.7.0/bin/MapAlignerPoseClustering -in {input} -out {output.var1} -trafo_out {output.var2}
            """

In [88]:
!resources/OpenMS-2.7.0/bin/MapAlignerPoseClustering -algorithm:max_num_peaks_considered -1 -in results/Epemicins/interim/MFD_Epemicins.featureXML results/GermicidinA/interim/MFD_GermicidinA.featureXML results/GermicidinB/interim/MFD_GermicidinB.featureXML -out results/Consensus/interim/MapAlignerPoseClustering_Epemicins.featureXML results/Consensus/interim/MapAlignerPoseClustering_GermicidinA.featureXML results/Consensus/interim/MapAlignerPoseClustering_GermicidinB.featureXML -trafo_out results/Consensus/interim/MapAlignerPoseClustering_Epemicins.trafoXML results/Consensus/interim/MapAlignerPoseClustering_GermicidinA.trafoXML results/Consensus/interim/MapAlignerPoseClustering_GermicidinB.trafoXML

Picking a reference (by size) ... done
Progress of 'Aligning input maps':
-- done [took 4.47 s (CPU), 4.54 s (Wall)] -- 
MapAlignerPoseClustering took 6.10 s (wall), 6.02 s (CPU), 0.30 s (system), 5.72 s (user); Peak Memory Usage: 68 MB.


#### Introduce the features to a protein identification file (idXML)   
Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/nightly/html/TOPP_IDMapper.html

    rule IDMapper:
        input:
            "resources/emptyfile.idXML",
            "results/Consensus/interim/MapAlignerPoseClustering_{samples}.featureXML",
            "results/{samples}/interim/precursorcorrected_{samples}.mzML"
        output:
            "results/Consensus/interim/IDMapper_{samples}.featureXML"
        shell:
            """
            resources/OpenMS-2.7.0/bin/IDMapper -id {input[0]} -in {input[1]}  -spectra:in {input[2]} -out {output} 
            """


In [ ]:
!resources/OpenMS-2.7.0/bin/IDMapper -id resources/emptyfile.idXML -in results/GNPSexport/interim/MapAlignerPoseClustering_Epemicins.featureXML  -spectra:in results/Epemicins/interim/precursorcorrected_Epemicins.mzML -out results/GNPSexport/interim/IDMapper_Epemicins.featureXML

#### The FeatureLinkerUnlabeledKD is used to aggregate the feature information (from single files) into a ConsensusFeature, linking features from different files together, which have a smiliar m/z and rt (no MS2 data).
Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/release/latest/html/TOPP_FeatureLinkerUnlabeledKD.html

    rule FeatureLinkerUnlabeledKD:
        input:
            expand("results/Consensus/interim/IDMapper_{samples}.featureXML", samples=SAMPLES)
        output:
            "results/Consensus/interim/FeatureLinkerUnlabeledKD.consensusXML"
        shell:
            """
            resources/OpenMS-2.7.0/bin/FeatureLinkerUnlabeledKD -in {input} -out {output} 
            """

In [ ]:
!resources/OpenMS-2.7.0/bin/FeatureLinkerUnlabeledKD -in results/Consensus/interim/IDMapper_Epemicins.featureXML results/Consensus/interim/IDMapper_GermicidinA.featureXML results/Consensus/interim/IDMapper_GermicidinB.featureXML -out results/Consensus/interim/FeatureLinkerUnlabeledKD.consensusXML

#### Filter out the features that do not have an MS2 pattern
Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/release/latest/html/TOPP_FileFilter.html

    rule FileFilter:
        input:
            "results/Consensus/interim/FeatureLinkerUnlabeledKD.consensusXML"
        output:
            "results/Consensus/filtered.consensusXML"
        shell:
            """
            resources/OpenMS-2.7.0/bin/FileFilter -id:remove_unannotated_features -in {input} -out {output} 
            """


In [ ]:
!resources/OpenMS-2.7.0/bin/FileFilter -id:remove_unannotated_features -in results/Consensus/interim/FeatureLinkerUnlabeledKD.consensusXML -out results/Consensus/filtered.consensusXML


#### GNPS_export creates an mgf file with only the MS2 information of all files (introduce mzml files with spaces between them)
Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/nightly/html/TOPP_GNPSExport.html

    rule GNPS_export:
        input:
            "results/Consensus/filtered.consensusXML",
            expand("results/{samples}/interim/precursorcorrected_{samples}.mzML", samples=SAMPLES)
        output:
            "results/GNPSexport/MSMS.mgf" 
        shell:
            """
            resources/OpenMS-2.7.0/bin/GNPSExport -in_cm {input[0]} -in_mzml {input[1]} -out {output} 
            """

In [ ]:
!resources/OpenMS-2.7.0/bin/GNPSExport -ini resources/GNPSExport.ini -in_cm results/Consensus/filtered.consensusXML -in_mzml results/Epemicins/interim/precursorcorrected_Epemicins.mzML results/GermicidinA/interim/precursorcorrected_GermicidinA.mzML results/GermicidinB/interim/precursorcorrected_GermicidinB.mzML -out results/GNPSexport/MSMS.mgf

Progress of 'parsing features and ms2 identifications...':
  1.63 %               GNPSExport took 2.62 s (wall), 2.58 s (CPU), 0.16 s (system), 2.42 s (user); Peak Memory Usage: 87 MB.


#### export the consensusXML file to a txt file for GNPS
Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/release/latest/html/TOPP_TextExporter.html
    
    rule txt_export:
        input:
            "results/Consensus/filtered.consensusXML"
        output:
            "results/GNPSexport/FeatureQuantificationTable.txt" 
        shell:
            """
            resources/OpenMS-2.7.0/bin/TextExporter -in {input} -out {output}
            """

In [ ]:
!resources/OpenMS-2.7.0/bin/TextExporter -in results/Consensus/filtered.consensusXML -out results/GNPSexport/FeatureQuantificationTable.txt

TextExporter took 0.09 s (wall), 0.09 s (CPU), 0.01 s (system), 0.08 s (user); Peak Memory Usage: 19 MB.


In [6]:
!resources/OpenMS-2.7.0/bin/FeatureFinderMetaboIdent -in results/Epemicins/interim/precursorcorrected_Epemicins.mzML -id MetaboliteIdentification.tsv -out results/Epemicins/interim/FFMI_Epemicins.featureXML

RT window size calculated as 240 seconds.
Loading targets and creating assay library...
Loading input LC-MS data...
Progress of 'loading spectra list':
-- done [took 0.48 s (CPU), 0.47 s (Wall)] -- 
Progress of 'loading chromatogram list':
-- done [took 0.00 s (CPU), 0.00 s (Wall)] -- 
Error: No sum formula given for target 'Unknown'; cannot calculate isotope distribution - using estimation method for peptides.
<RT window size calculated as 240 seconds.> occurred 4 times
Extracting chromatograms...
Detecting chromatographic peaks...
Found no signal. The Gaussian width is probably smaller than the spacing in your chromatogram data. Try to use a bigger width. The error occurred in the chromatogram with m/z time 561.357276466771054.
Found no signal. The Gaussian width is probably smaller than the spacing in your chromatogram data. Try to use a bigger width. The error occurred in the chromatogram with m/z time 562.360631304571029.
Found no signal. The Gaussian width is probably smaller tha

In [15]:
from collections import defaultdict
from functools import reduce
from pathlib import Path
from time import perf_counter
import sys

from IPython.core.display import display
from pandas import CategoricalDtype
import numpy as np
from pyopenms import *
import pandas as pd
import os

class FeatureMapDF(FeatureMap):      
    def __init__(self):
        super().__init__()

    def get_fmi_df(self):
        def gen(fmap: FeatureMap, fun):
            for f in fmap:
                yield from fun(f)

        def extractMetaData(f: Feature):
            metavals = []
            types = []
            for k in metavals:
                if k == b"MZ":
                    types.append('f')
                elif k== b"PeptideRef":
                    types.append('U100')
                elif k== b"masserror_ppm":
                    types.append('i4')
                elif k== b"masstrace_intensity":
                    types.append("f")
            # subfeatures = f.getFeatureList()  # type: list[FeatureHandle]
            pep = f.getPeptideIdentifications()  # type: list[PeptideIdentification]
            bb = f.getConvexHull().getBoundingBox2D()
            for u in metavals:
                f.getMetaValue(u)
            if len(pep) != 0:
                hits = pep[0].getHits()
                if len(hits) != 0:
                    besthit = hits[0]  # type: PeptideHit
                    yield f.getUniqueId(), besthit.getSequence().toString(), f.getCharge(), f.getRT(), f.getMZ(), bb[0][0], bb[1][0], f.getMetaValue("PeptideRef"), f.getOverallQuality(), f.getIntensity()
                else:
                    yield f.getUniqueId(), None, f.getCharge(), f.getRT(), f.getMZ(), bb[0][0], bb[1][0], f.getMetaValue("PeptideRef"), f.getOverallQuality(), f.getIntensity()
            else:
                yield f.getUniqueId(), None, f.getCharge(), f.getRT(), f.getMZ(), bb[0][0], bb[1][0], f.getMetaValue("PeptideRef"), f.getOverallQuality(), f.getIntensity()

        cnt = self.size()

        mddtypes = [('id', np.dtype('uint64')), ('sequence', 'U200'), ('charge', 'i4'), ('RT', 'f'), ('mz', 'f'),
                    ('RTstart', 'f'), ('RTend', 'f'), ("PeptideRef", 'U100'),
                    ('quality', 'f'), ('intensity', 'f')]
        mdarr = np.fromiter(iter=gen(self, extractMetaData), dtype=mddtypes, count=cnt)
        df= pd.DataFrame(mdarr).set_index('id').sort_values("mz").drop(columns= "sequence")
        return df

In [16]:
filename= './results/Epemicins/interim/FFMI_Epemicins.featureXML'
fmap = FeatureMapDF()
FeatureXMLFile().load(filename, fmap)
DF= fmap.get_fmi_df()
feature_csv= './results/Epemicins/FFMI_Epemicins.csv'
DF.to_csv(feature_csv)
display(DF)

,charge,RT,mz,RTstart,RTend,PeptideRef,quality,intensity
id,,,,,,,,
6540659878518515890,1,46.026489,402.207275,37.475689,54.632912,bioxalomycina2_m401.200012_z1_rt48.18,-2.825665,2.926461e+04
280489307960157370,1,103.041199,474.207275,91.933479,113.822960,Unknown_m473.200012_z1_rt44.200001,-0.763105,2.209964e+05
9143164355464892809,1,265.116608,566.277283,243.804138,286.463776,Coelichelin_m565.27002_z1_rt268.049988,-7.000911,1.934900e+04
10815806269762203077,1,373.222076,585.357300,353.652527,393.302399,dehydroxynocardamine_m584.349976_z1_rt351.070007,-7.667008,6.850766e+05
14786621585738311964,1,286.942780,587.337280,275.987885,310.614807,desmethylenylnocardamine_m586.330017_z1_rt339....,-7.992048,9.937172e+04
10021272275670129286,1,298.578583,601.357300,259.177887,319.175507,desferrioxamineE_m600.349976_z1_rt329.5,-10.806923,1.364715e+05
9181399712805582150,2,379.947540,712.417297,333.265137,412.114746,epemicinB_m1422.819946_z2_rt405.769989,4.456927,9.244367e+09
11025134690498236929,2,367.807312,793.447266,326.792847,404.532959,epemicinA_m1584.880005_z2_rt386.850006,4.391304,3.584970e+09
12247998505257479685,1,563.042603,797.417297,537.345642,597.198547,Kirromycin_m796.409973_z1_rt618.98999,3.953642,1.193821e+06
